### OCI Data Science - pull data
This notebook provides toos and techniques to pull required data for the project

##### Use fastF1 API 
* Pull Lap, weather, car, etc data 




#### This notebook uses formula1conda which is custom conda
fastf1 package requires python>=3.8 and OCI prebuild packages come with python==3.7.

In [2]:
path = '/home/datascience/WorkSpace/RedBull-Racining-TimeToPit/notebooks'

In [31]:
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import fastf1
import matplotlib.pyplot as plt
import pickle
fastf1.Cache.enable_cache('../../RedBull/data')

In [38]:
schedule = fastf1.get_event_schedule(2022).to_dict()
schedule = pd.DataFrame.from_dict(schedule)

schedule.drop(schedule[schedule['EventName'] == 'Pre-Season Test'].index, inplace = True)
schedule
# len(np.unique(schedule['EventName']) ), schedule['EventName']

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
0,0,Spain,Spain,FORMULA 1 PRE-SEASON TRACK SESSION 2022,2022-02-25 18:00:00,Pre-Season Track Session,testing,Practice 1,2022-02-23 09:00:00,Practice 2,2022-02-24 09:00:00,Practice 3,2022-02-25 09:00:00,None,NaT,None,NaT,False
2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,2022-03-20 20:00:00,Bahrain Grand Prix,conventional,Practice 1,2022-03-18 15:00:00,Practice 2,2022-03-18 18:00:00,Practice 3,2022-03-19 15:00:00,Qualifying,2022-03-19 18:00:00,Race,2022-03-20 18:00:00,True
3,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,2022-03-27 22:00:00,Saudi Arabian Grand Prix,conventional,Practice 1,2022-03-25 17:00:00,Practice 2,2022-03-25 20:00:00,Practice 3,2022-03-26 17:00:00,Qualifying,2022-03-26 20:00:00,Race,2022-03-27 20:00:00,True
4,3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,2022-04-10 17:00:00,Australian Grand Prix,conventional,Practice 1,2022-04-08 13:00:00,Practice 2,2022-04-08 16:00:00,Practice 3,2022-04-09 13:00:00,Qualifying,2022-04-09 16:00:00,Race,2022-04-10 15:00:00,True
5,4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...,2022-04-24 17:00:00,Emilia Romagna Grand Prix,sprint,Practice 1,2022-04-22 13:30:00,Qualifying,2022-04-22 17:00:00,Practice 2,2022-04-23 12:30:00,Sprint,2022-04-23 16:30:00,Race,2022-04-24 15:00:00,True
6,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,2022-05-08 17:30:00,Miami Grand Prix,conventional,Practice 1,2022-05-06 14:30:00,Practice 2,2022-05-06 17:30:00,Practice 3,2022-05-07 13:00:00,Qualifying,2022-05-07 16:00:00,Race,2022-05-08 15:30:00,True
7,6,Spain,Barcelona,FORMULA 1 PIRELLI GRAN PREMIO DE ESPAÑA 2022,2022-05-22 17:00:00,Spanish Grand Prix,conventional,Practice 1,2022-05-20 14:00:00,Practice 2,2022-05-20 17:00:00,Practice 3,2022-05-21 13:00:00,Qualifying,2022-05-21 16:00:00,Race,2022-05-22 15:00:00,True
8,7,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2022,2022-05-29 17:00:00,Monaco Grand Prix,conventional,Practice 1,2022-05-27 14:00:00,Practice 2,2022-05-27 17:00:00,Practice 3,2022-05-28 13:00:00,Qualifying,2022-05-28 16:00:00,Race,2022-05-29 15:00:00,True
9,8,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2022,2022-06-12 17:00:00,Azerbaijan Grand Prix,conventional,Practice 1,2022-06-10 15:00:00,Practice 2,2022-06-10 18:00:00,Practice 3,2022-06-11 15:00:00,Qualifying,2022-06-11 18:00:00,Race,2022-06-12 15:00:00,True
10,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2022,2022-06-19 16:00:00,Canadian Grand Prix,conventional,Practice 1,2022-06-17 14:00:00,Practice 2,2022-06-17 17:00:00,Practice 3,2022-06-18 13:00:00,Qualifying,2022-06-18 16:00:00,Race,2022-06-19 14:00:00,True


In [51]:
evnt

'Austrian Grand Prix'

In [ ]:
%%time
for yrs in [2020, 2019]:
    laps =[]
    weathers =[]
    results = []
    car_data_ses =[]
    all_positions =[]
    
    schedule = fastf1.get_event_schedule(yrs).to_dict()
    schedule = pd.DataFrame.from_dict(schedule)

#     schedule.drop(schedule[schedule['EventName'] == 'Pre-Season Test'].index, inplace = True)
#     EventName = list(schedule['EventName'])
    EventName = [s for s in schedule['EventName'] if "Grand" in s]
    
    for evnt in EventName:
        for ses in ['Qualifying','Race']:
            session = fastf1.get_session(yrs, evnt, ses)
            session.load()
            
            lap = session.laps.to_dict()
            lap = pd.DataFrame.from_dict(lap)
            lap['EventName'] =evnt
            lap['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            lap['session'] = ses
            lap['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]
            
            weather = session.weather_data.to_dict()
            weather = pd.DataFrame.from_dict(weather)
            weather['EventName'] =evnt
            weather['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            weather['session'] = ses
            weather['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]

            
            for ii in session.car_data:
                car_data = session.car_data[ii].to_dict()
                car_data = pd.DataFrame.from_dict(car_data)
                car_data['driver'] = ii
            car_data['EventName'] =evnt
            car_data['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            car_data['session'] = ses
            car_data['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]
            
            for ii in session.pos_data:
                position = session.pos_data[ii].to_dict()
                position = pd.DataFrame.from_dict(position)
                position['driver'] = ii
            position['EventName'] =evnt
            position['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            position['session'] = ses
            position['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]    
                
            result = session.results.to_dict()
            result = pd.DataFrame.from_dict(result).reset_index()
            result['EventName'] =evnt
            result['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
            result['session'] = ses
            result['EventDate'] = schedule['EventDate'][schedule['EventName']==evnt].values[0]
            
            laps.append(lap)
            weathers.append(weather)
            results.append(result)
            car_data_ses.append(car_data)
            all_positions.append(position) 
    
    file = open('../../RedBull/data/laps_'+str(yrs)+'.pkl', 'wb')
    pickle.dump(laps,file)
    file = open('../../RedBull/data/weathers'+str(yrs)+'.pkl', 'wb')
    pickle.dump(weathers,file)
    file = open('../../RedBull/data/results'+str(yrs)+'.pkl', 'wb')
    pickle.dump(results,file)
    file = open('../../RedBull/data/car_data_ses'+str(yrs)+'.pkl', 'wb')
    pickle.dump(car_data_ses,file)
    file = open('../../RedBull/data/all_positions'+str(yrs)+'.pkl', 'wb')
    pickle.dump(all_positions,file)

core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v2.2.8]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_app_data. Loading data...
api            INFO 	Fetching timing app data...
api            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
api            INFO 	No cached data found for session_status_data. Loading data...
api            INFO 	Fetching session status data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for track_status_data. Loading data...
api            INFO 	Fetching tr